In [21]:
import plotly
from plotly.graph_objs import Scatter, Layout
import numpy as np
import tensorflow as tf
import sys
plotly.offline.init_notebook_mode(connected=True)

In [22]:
sample_length = 50001
time_per_sample = 0.01
signal_time = np.linspace(num=sample_length,start = 0, stop = sample_length * time_per_sample )
signal_amp = np.sin(signal_time*2*np.pi) + \
    np.sin(2+signal_time*1.7*np.pi)*0.5 + \
    np.sin(1+signal_time*2.2*np.pi) + \
    np.random.normal(size=sample_length)*0.02

In [23]:
s_i = 0
e_i = s_i + 100
plotly.offline.iplot({
    "data": [Scatter(x=signal_time[s_i:e_i],y=signal_amp[s_i:e_i])],
    "layout": Layout(title="")
})

In [24]:
sequence_length = 50
prediction_length = 1
input_feature_count = 1
output_feature_count = 1
batch_size = 512
hidden_count_per_layer = [50,50]

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [batch_size, sequence_length, input_feature_count], name = 'inputs')
targets = tf.placeholder(tf.float32, [batch_size, output_feature_count], name = 'targets')
keep_prob = tf.placeholder(tf.float32, name = 'keep')
learning_rate = tf.placeholder(tf.float32, name = 'learning_rate')

In [25]:
layers = []


for hidden_count in hidden_count_per_layer:
    layer =  tf.nn.rnn_cell.LSTMCell(hidden_count, state_is_tuple=True)
    layer_with_dropout = tf.nn.rnn_cell.DropoutWrapper(layer,
                                          input_keep_prob=keep_prob,
                                          output_keep_prob=1.0)
    layers.append(layer)
hidden_network = tf.nn.rnn_cell.MultiRNNCell(layers, state_is_tuple=True)   



In [26]:
def get_network_state_size(network,batch_size):
    states = 0
    for layer_size in hidden_network.state_size:
        states += layer_size[0]
        states += layer_size[1]
    return states * batch_size
    
def pack_state_tuple(states_per_layer):
    return tf.concat(axis=0, values=[tf.reshape(layer_states, (-1,1)) for layer_states in states_per_layer])

def unpack_state_tuple(network, batch_size, packed_states):
    rnn_list_state = []
    start_index = 0

    for layer_size in hidden_network.state_size:
            states_cnt = batch_size * layer_size[0]
            #print("layer",layer_size)
            # add a tuple with 2 tensors (batch_size, hidden_count )
            #print("c", packed_states.shape, start_index, states_cnt)
            states_c = tf.slice(packed_states, begin=[start_index,0], size=[states_cnt,1])
            states_c = tf.reshape(states_c,[batch_size, layer_size[0]])
            start_index += states_cnt
            states_cnt = batch_size * layer_size[1]
            #print("h", packed_states.shape, start_index, states_cnt)
            states_h = tf.slice(initial_state, begin=[start_index,0], size=[states_cnt,1])
            states_h = tf.reshape(states_h,[batch_size, layer_size[1]])
            start_index += states_cnt
            
            rnn_list_state.append(tf.nn.rnn_cell.LSTMStateTuple(states_c,states_h))
    tuple_state=tuple(rnn_list_state)            
    return tuple_state

In [27]:
sz = get_network_state_size(hidden_network,batch_size)
print("states in network", sz)

zero_state = pack_state_tuple(hidden_network.zero_state(batch_size, tf.float32))
initial_state = tf.placeholder_with_default(
    zero_state, 
    (sz,1), 
    name="initial_state")



states in network 102400


In [28]:
# Test to check that the pack and unpack functions are eachothers inverse

#copy of the input (packed) shape (x,1)
initial_copy = tf.slice(initial_state,(0,0),(sz,1))

#unpack into (LSTMTuple(),LSTMTuple(),....)
unpacked = unpack_state_tuple(hidden_network,batch_size,initial_copy)
rnn_tuple_state = unpacked
#repack into vector (x,1)
packed_again = pack_state_tuple(unpacked)


init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    #v_outputs, v_state = sess.run([outputs,state], feed_dict={inputs: batch_inputs, targets: batch_targets})
    cp = sess.run(initial_copy,  feed_dict={}) #initial_state: initial_state_input})
    #cp[0] = 1
    cp[:,0] = np.linspace(start=0,stop=cp.shape[0],num=cp.shape[0])
    
    cp,up,cp2 = sess.run([initial_copy,unpacked,packed_again],  feed_dict={initial_state: cp}) #initial_state: initial_state_input})
    #print("cp",cp.shape,cp)
    #print("up",unpacked,up)
    #print("cp2",packed_again,cp2)
    #print("up2",unpacked_again,up2)
    diff = cp - cp2
    print("diff",np.sum(diff))

diff 0.0


In [29]:
#out_weights=tf.Variable(tf.random_normal([hidden_count_per_layer[-1],output_feature_count]))
#out_bias=tf.Variable(tf.random_normal([output_feature_count]))
print("inputs ",inputs.shape)
outputs, state = tf.nn.dynamic_rnn(hidden_network, inputs, dtype=tf.float32, initial_state=rnn_tuple_state, )
state_packed = pack_state_tuple(state)
print("outputs before transpose", outputs.shape)
outputs = tf.transpose(outputs, [1, 0, 2])
print("outputs after transpose", outputs.shape)
last_output = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)
print("last output", last_output.shape)
                                   
#out_size = target.get_shape()[2].value
predictions = tf.contrib.layers.fully_connected(last_output, output_feature_count, activation_fn=None)
print("prediction", predictions.shape)
print("targets", targets.shape)
#prediction = tf.nn.softmax(logit)
#loss = tf.losses.softmax_cross_entropy(target, logit)


inputs  (512, 50, 1)
outputs before transpose (512, 50, 50)
outputs after transpose (50, 512, 50)
last output (512, 50)
prediction (512, 1)
targets (512, 1)


In [30]:
loss = tf.reduce_sum(tf.squared_difference(predictions, targets))

#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [31]:
start_indices = np.linspace(
    0,
    sample_length-sequence_length-prediction_length-1,
    sample_length-sequence_length-prediction_length-1, dtype= np.int32)

dev_size_perc = 0.20
test_size_perc = 0.20

dev_size = int(np.floor(start_indices.shape[0] * dev_size_perc))
test_size  = int(np.floor(start_indices.shape[0] * test_size_perc))
train_size = start_indices.shape[0] - test_size - dev_size
train_batch_count = int(np.floor(train_size / batch_size))
dev_batch_count = int(np.floor(dev_size / batch_size))
test_batch_count = int(np.floor(test_size / batch_size))

print("dataset size %d" %(start_indices.shape[0]))
print("%d Examples (%d batches) in train set" %(train_size, train_batch_count))
print("%d Examples (%d batches) in dev set" %(dev_size,dev_batch_count))
print("%d Examples (%d batches) in test set" %(test_size,test_batch_count))



dataset size 49949
29971 Examples (58 batches) in train set
9989 Examples (19 batches) in dev set
9989 Examples (19 batches) in test set


In [32]:
np.random.shuffle (start_indices)
train_indices = start_indices[0:int(train_size)]
dev_indices= start_indices[int(train_size):int(train_size+dev_size)]
test_indices = start_indices[int(train_size+dev_size):int(train_size+dev_size+test_size)]

def get_batch(batch_index, indexes, size=batch_size):
    batch_start_indexes = indexes[batch_index*size:batch_index*size+size]
    batch_inputs = np.zeros((size,sequence_length, input_feature_count))
    batch_targets = np.zeros((size,prediction_length))
    for i in range(size):
        se = batch_start_indexes[i]
        part = signal_amp[se:se+sequence_length]
        batch_inputs[i,0:sequence_length,0] = part
        batch_targets[i,0] = signal_amp[se+sequence_length+1]

    return batch_inputs,batch_targets

batch_inputs,batch_targets = get_batch(train_batch_count-1,train_indices)
print(batch_inputs.shape,batch_targets.shape)

example_inputs = batch_inputs[0,:,:]
example_targets =  batch_targets[0,:]
print(example_inputs.shape)
#b_i = 1
#b_s = batch_inputs[b_i,0:sequence_length,0]
#plotly.offline.iplot({
#    "data": [Scatter(y=b_s)],
#    "layout": Layout(title="")
#})

(512, 50, 1) (512, 1)
(50, 1)


In [75]:
np.random.shuffle (start_indices)
train_indices = start_indices[0:int(train_size)]
dev_indices= start_indices[int(train_size):int(train_size+dev_size)]
test_indices = start_indices[int(train_size+dev_size):int(train_size+dev_size+test_size)]


init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    np.random.shuffle (train_indices)
    
    batch_inputs,batch_targets = get_batch(0, train_indices)
    print("batch input shape", batch_inputs.shape)
    #v_outputs, v_state = sess.run([outputs,state], feed_dict={inputs: batch_inputs, targets: batch_targets})
    v_predictions, v_state, v_state_packed = sess.run([predictions,state, state_packed], 
                                      feed_dict={
                                          inputs: batch_inputs, 
                                          targets: batch_targets
                                      })
    print(v_predictions.shape)
    print(v_predictions[0],batch_targets[0])
    for i in range(0,120):
        v_predictions, v_outputs, v_state, v_state_packed, v_loss, v_opt = sess.run(
            [predictions, outputs, state, state_packed, loss, opt], 
            feed_dict={
                learning_rate: 0.02, 
                inputs: batch_inputs, 
                targets: batch_targets,
                initial_state: v_state_packed
            }) #})
        print(v_loss,v_predictions[0],batch_targets[0])
 

    
    

batch input shape (512, 50, 1)
(512, 1)
[-0.11653353] [ 0.58121891]
288.75 [-0.11647449] [ 0.58121891]
225.513 [ 0.07935123] [ 0.58121891]
177.012 [ 0.20965971] [ 0.58121891]
129.561 [ 0.38865343] [ 0.58121891]
96.3236 [ 0.60352397] [ 0.58121891]
66.3575 [ 0.72608513] [ 0.58121891]
35.9544 [ 0.69287181] [ 0.58121891]
16.9334 [ 0.62053478] [ 0.58121891]
11.0414 [ 0.612423] [ 0.58121891]
8.85931 [ 0.67310977] [ 0.58121891]
16.8448 [ 0.82962584] [ 0.58121891]
13.4691 [ 0.70581067] [ 0.58121891]
11.2271 [ 0.78637755] [ 0.58121891]
7.71416 [ 0.58813578] [ 0.58121891]
4.58144 [ 0.66887516] [ 0.58121891]
6.86396 [ 0.78906053] [ 0.58121891]
6.8814 [ 0.73353046] [ 0.58121891]
6.7304 [ 0.67206466] [ 0.58121891]
5.39651 [ 0.68908876] [ 0.58121891]
5.03756 [ 0.66945201] [ 0.58121891]
3.43868 [ 0.60664016] [ 0.58121891]
2.35383 [ 0.60541523] [ 0.58121891]
1.7797 [ 0.66800594] [ 0.58121891]
1.87366 [ 0.67074525] [ 0.58121891]
1.48717 [ 0.59801883] [ 0.58121891]
1.27699 [ 0.56348521] [ 0.58121891]
1.

In [34]:
np.random.shuffle (start_indices)
train_indices = start_indices[0:int(train_size)]
dev_indices= start_indices[int(train_size):int(train_size+dev_size)]
test_indices = start_indices[int(train_size+dev_size):int(train_size+dev_size+test_size)]

epoch_count = 20

loss_results = np.zeros((epoch_count,2))

def get_dev_loss():
    epoch_dev_loss = 0.0
    for devi in range(dev_batch_count):
        batch_inputs,batch_targets = get_batch(devi, dev_indices)

        batch_dev_loss = sess.run(loss,feed_dict={inputs:batch_inputs,targets:batch_targets})
        print("  Dev results batch %d, loss %s" %(  devi, str(batch_dev_loss)))  

        epoch_dev_loss += batch_dev_loss
        #sys.stdout.write('.')
        #sys.stdout.flush()
    return epoch_dev_loss / dev_size

def generate_graph():
    graph_size = 200
    prime_size = 20
    
    prime_signal_start_i = 0
    
    tmp_signal = np.zeros((graph_size,1))
    tmp_signal[0:prime_size,0] = signal_amp[prime_signal_start_i:(prime_signal_start_i+prime_size)]
    
    tmp_batch = np.zeros((batch_size,sequence_length,1))
    print(tmp_batch.shape)
    print(inputs.shape)
    p_i_e = prime_size
    p_i_s = p_i_e - sequence_length
    
    for end in range(prime_size, graph_size):
        #end = prime_size
        tmp_batch[0,:,0] = tmp_signal.take(range((end-sequence_length),end), mode='wrap')
        _predictions = sess.run([predictions[0,0]], feed_dict={learning_rate: 0.02, inputs: tmp_batch})
        #print(_predictions)
        tmp_signal[end,0] = _predictions[0]
        sys.stdout.write('.')
        sys.stdout.flush()
    print("")
    
    plotly.offline.iplot({
        "data": [Scatter(y=tmp_signal[:,0])],
        "layout": Layout(title="")
    })


init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    epoch_dev_loss = get_dev_loss()    

    #print("")            
    print("Dev results epoch start, loss %s" %(  str(epoch_dev_loss),))  

    for epoch in range(0,epoch_count):
        np.random.shuffle (train_indices)
        epoch_train_loss = 0.0
        for ti in range(train_batch_count):
            batch_inputs,batch_targets = get_batch(ti, train_indices)

            batch_train_loss, _ = sess.run([loss, opt], feed_dict={learning_rate: 0.02, inputs: batch_inputs, targets: batch_targets})
            print("  Train results batch %d, loss %s" %(  ti, str(batch_train_loss)))  
            epoch_train_loss += batch_train_loss
            #sys.stdout.write('.')
            #sys.stdout.flush()
        #print("")
        epoch_train_loss = epoch_train_loss / train_size
        print("Training results epoch %d, loss %s" %( epoch, str(epoch_train_loss)))
        epoch_dev_loss = get_dev_loss()    
        #print("")            
        print("Dev results epoch %d, loss %s" %( epoch, str(epoch_dev_loss)))  
        loss_results[epoch,0] = epoch_train_loss
        loss_results[epoch,1] = epoch_dev_loss
        ti += 1
        #generate_graph()
         

  Dev results batch 0, loss 458.682
  Dev results batch 1, loss 525.952
  Dev results batch 2, loss 519.397
  Dev results batch 3, loss 475.219
  Dev results batch 4, loss 558.162
  Dev results batch 5, loss 449.233
  Dev results batch 6, loss 510.851
  Dev results batch 7, loss 514.147
  Dev results batch 8, loss 508.348
  Dev results batch 9, loss 503.836
  Dev results batch 10, loss 496.116
  Dev results batch 11, loss 508.366
  Dev results batch 12, loss 492.309
  Dev results batch 13, loss 512.456
  Dev results batch 14, loss 503.431
  Dev results batch 15, loss 514.911
  Dev results batch 16, loss 510.841
  Dev results batch 17, loss 544.195
  Dev results batch 18, loss 485.198
Dev results epoch start, loss 0.960221178599
  Train results batch 0, loss 493.044
  Train results batch 1, loss 537.421
  Train results batch 2, loss 492.49
  Train results batch 3, loss 317.076
  Train results batch 4, loss 218.079
  Train results batch 5, loss 150.922
  Train results batch 6, loss 43.37

  Train results batch 33, loss 0.439899
  Train results batch 34, loss 0.42923
  Train results batch 35, loss 0.479
  Train results batch 36, loss 0.37423
  Train results batch 37, loss 0.386146
  Train results batch 38, loss 0.450983
  Train results batch 39, loss 0.387242
  Train results batch 40, loss 0.41585
  Train results batch 41, loss 0.435805
  Train results batch 42, loss 0.443473
  Train results batch 43, loss 0.425311
  Train results batch 44, loss 0.406952
  Train results batch 45, loss 0.399488
  Train results batch 46, loss 0.382407
  Train results batch 47, loss 0.393976
  Train results batch 48, loss 0.444474
  Train results batch 49, loss 0.43777
  Train results batch 50, loss 0.389165
  Train results batch 51, loss 0.416331
  Train results batch 52, loss 0.438589
  Train results batch 53, loss 0.439667
  Train results batch 54, loss 0.428943
  Train results batch 55, loss 0.443978
  Train results batch 56, loss 0.439431
  Train results batch 57, loss 0.398493
Trainin

  Train results batch 6, loss 0.395297
  Train results batch 7, loss 0.378347
  Train results batch 8, loss 0.356214
  Train results batch 9, loss 0.387321
  Train results batch 10, loss 0.420265
  Train results batch 11, loss 0.401444
  Train results batch 12, loss 0.480586
  Train results batch 13, loss 0.426355
  Train results batch 14, loss 0.416635
  Train results batch 15, loss 0.41679
  Train results batch 16, loss 0.411181
  Train results batch 17, loss 0.3828
  Train results batch 18, loss 0.428458
  Train results batch 19, loss 0.389428
  Train results batch 20, loss 0.387386
  Train results batch 21, loss 0.418089
  Train results batch 22, loss 0.399487
  Train results batch 23, loss 0.349103
  Train results batch 24, loss 0.4141
  Train results batch 25, loss 0.421721
  Train results batch 26, loss 0.41958
  Train results batch 27, loss 0.412013
  Train results batch 28, loss 0.417829
  Train results batch 29, loss 0.337066
  Train results batch 30, loss 0.388976
  Train re

  Dev results batch 6, loss 0.433113
  Dev results batch 7, loss 0.453265
  Dev results batch 8, loss 0.458946
  Dev results batch 9, loss 0.473936
  Dev results batch 10, loss 0.479998
  Dev results batch 11, loss 0.44028
  Dev results batch 12, loss 0.389614
  Dev results batch 13, loss 0.472066
  Dev results batch 14, loss 0.516823
  Dev results batch 15, loss 0.505862
  Dev results batch 16, loss 0.48869
  Dev results batch 17, loss 0.452845
  Dev results batch 18, loss 0.407077
Dev results epoch 7, loss 0.000882156541582
  Train results batch 0, loss 0.452698
  Train results batch 1, loss 0.424393
  Train results batch 2, loss 0.479554
  Train results batch 3, loss 0.442627
  Train results batch 4, loss 0.454932
  Train results batch 5, loss 0.387547
  Train results batch 6, loss 0.374886
  Train results batch 7, loss 0.47603
  Train results batch 8, loss 0.528676
  Train results batch 9, loss 0.397058
  Train results batch 10, loss 0.389932
  Train results batch 11, loss 0.489365

  Train results batch 40, loss 0.491933
  Train results batch 41, loss 0.38102
  Train results batch 42, loss 0.469212
  Train results batch 43, loss 0.486724
  Train results batch 44, loss 0.37486
  Train results batch 45, loss 0.384591
  Train results batch 46, loss 0.412796
  Train results batch 47, loss 0.381522
  Train results batch 48, loss 0.438613
  Train results batch 49, loss 0.351587
  Train results batch 50, loss 0.497761
  Train results batch 51, loss 0.547874
  Train results batch 52, loss 0.394701
  Train results batch 53, loss 0.378122
  Train results batch 54, loss 0.435158
  Train results batch 55, loss 0.434772
  Train results batch 56, loss 0.372886
  Train results batch 57, loss 0.401503
Training results epoch 10, loss 0.000812377733319
  Dev results batch 0, loss 0.541698
  Dev results batch 1, loss 0.496569
  Dev results batch 2, loss 0.548289
  Dev results batch 3, loss 0.520897
  Dev results batch 4, loss 0.583666
  Dev results batch 5, loss 0.514405
  Dev resu

  Train results batch 12, loss 0.380815
  Train results batch 13, loss 0.661096
  Train results batch 14, loss 0.775604
  Train results batch 15, loss 0.461807
  Train results batch 16, loss 0.496363
  Train results batch 17, loss 0.700317
  Train results batch 18, loss 0.38121
  Train results batch 19, loss 0.576233
  Train results batch 20, loss 0.729822
  Train results batch 21, loss 0.400877
  Train results batch 22, loss 0.629935
  Train results batch 23, loss 0.620508
  Train results batch 24, loss 0.44061
  Train results batch 25, loss 0.97519
  Train results batch 26, loss 0.90961
  Train results batch 27, loss 0.400017
  Train results batch 28, loss 0.628799
  Train results batch 29, loss 0.661312
  Train results batch 30, loss 0.493226
  Train results batch 31, loss 0.500193
  Train results batch 32, loss 0.644779
  Train results batch 33, loss 0.40652
  Train results batch 34, loss 0.544628
  Train results batch 35, loss 0.574734
  Train results batch 36, loss 0.419392
  Tra

  Dev results batch 4, loss 0.598256
  Dev results batch 5, loss 0.496413
  Dev results batch 6, loss 0.524085
  Dev results batch 7, loss 0.51534
  Dev results batch 8, loss 0.520979
  Dev results batch 9, loss 0.540719
  Dev results batch 10, loss 0.528674
  Dev results batch 11, loss 0.471635
  Dev results batch 12, loss 0.470365
  Dev results batch 13, loss 0.53413
  Dev results batch 14, loss 0.558848
  Dev results batch 15, loss 0.573649
  Dev results batch 16, loss 0.537475
  Dev results batch 17, loss 0.523115
  Dev results batch 18, loss 0.457577
Dev results epoch 15, loss 0.0009915017189
  Train results batch 0, loss 0.544592
  Train results batch 1, loss 0.379524
  Train results batch 2, loss 0.400014
  Train results batch 3, loss 0.475495
  Train results batch 4, loss 0.416946
  Train results batch 5, loss 0.384585
  Train results batch 6, loss 0.43549
  Train results batch 7, loss 0.35341
  Train results batch 8, loss 0.4311
  Train results batch 9, loss 0.415991
  Train r

  Train results batch 38, loss 0.443852
  Train results batch 39, loss 0.426562
  Train results batch 40, loss 0.464662
  Train results batch 41, loss 0.435727
  Train results batch 42, loss 0.666667
  Train results batch 43, loss 0.577548
  Train results batch 44, loss 0.547577
  Train results batch 45, loss 0.602061
  Train results batch 46, loss 0.838545
  Train results batch 47, loss 0.461047
  Train results batch 48, loss 0.842091
  Train results batch 49, loss 0.501634
  Train results batch 50, loss 0.812919
  Train results batch 51, loss 1.16382
  Train results batch 52, loss 0.512962
  Train results batch 53, loss 0.548954
  Train results batch 54, loss 1.02244
  Train results batch 55, loss 0.549562
  Train results batch 56, loss 0.477284
  Train results batch 57, loss 0.939979
Training results epoch 18, loss 0.000962127279636
  Dev results batch 0, loss 0.532525
  Dev results batch 1, loss 0.630548
  Dev results batch 2, loss 0.532717
  Dev results batch 3, loss 0.53229
  Dev

In [28]:
plotly.offline.iplot({
    "data": [Scatter(y=loss_results[2:,0]),Scatter(y=loss_results[2:,1])],
    "layout": Layout(title="")
})
plotly.offline.iplot
plotly.offline.iplot({
    "data": [Scatter(y=loss_results[2:,0]),Scatter(y=loss_results[2:,1])],
    "layout": Layout(title="")
})


In [39]:

generate_graph()
